# dcGAN

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def build_generator():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(100,)), 
        layers.Dense(7 * 7 * 256, use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model


In [12]:
def build_discriminator():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(28, 28, 1)), 
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model


In [13]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  

buffer_size = 60000
batch_size = 256
noise_dim = 100
num_examples_to_generate = 16

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

generator = build_generator()
discriminator = build_discriminator()

dummy_noise = tf.random.normal([1, 100])
_ = generator(dummy_noise)

dummy_image = tf.random.normal([1, 28, 28, 1])
_ = discriminator(dummy_image)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [15]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [16]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

generator = build_generator()
discriminator = build_discriminator()

dummy_noise = tf.random.normal([1, 100])
_ = generator(dummy_noise)

dummy_image = tf.random.normal([1, 28, 28, 1])
_ = discriminator(dummy_image)

train(train_dataset, epochs=50)


ValueError: in user code:

    File "C:\Users\filip\AppData\Local\Temp\ipykernel_13768\3214791541.py", line 17, in train_step  *
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    File "C:\Users\filip\miniconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 291, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "C:\Users\filip\miniconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 332, in apply
        self._check_variables_are_known(trainable_variables)
    File "C:\Users\filip\miniconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 237, in _check_variables_are_known
        raise ValueError(

    ValueError: Unknown variable: <KerasVariable shape=(100, 12544), dtype=float32, path=sequential_8/dense_8/kernel>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should r